# Rede Neural

In [ ]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-6')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# Visão Computacional

In [ ]:
base = 3
modo = "KMEANS"

if(base == 1):
    
    PASTA_RESULTADO = "FOLHAS_SEM_RESULTADOS_" + modo + "/"
    PASTA_RESULTADOS_ARQUIVOS = "DADOS_RESULTADOS/"
    PASTA = "RESULTADOS/"
    IMAGENS_VALIDACAO_PASTA = "RESULTADOS/FOLHAS_SEM/"
    IMAGENS_VALIDACAO_ARQUIVO = "_manga_sem_folha.png"

    qtdImagens = 249

elif(base == 2):
    
    PASTA_RESULTADO = "FOLHAS_COM_RESULTADOS_" + modo + "/"
    PASTA_RESULTADOS_ARQUIVOS = "DADOS_RESULTADOS/"
    PASTA = "RESULTADOS/"
    IMAGENS_VALIDACAO_PASTA = "RESULTADOS/FOLHAS_COM/"
    IMAGENS_VALIDACAO_ARQUIVO = "_manga_com_folha.png"
    
    qtdImagens = 208
    
elif(base == 3):
    
    PASTA_RESULTADO = "BASE_ANTIGA_VALIDACAO_RESULTADOS_" + modo + "/"
    PASTA_RESULTADOS_ARQUIVOS = "DADOS_RESULTADOS/"
    PASTA = "RESULTADOS/"
    IMAGENS_VALIDACAO_PASTA = "RESULTADOS/BASE_ANTIGA_VALIDACAO/"
    IMAGENS_VALIDACAO_ARQUIVO = "_manga_base_antiga.png"
    
    qtdImagens = 106
    
elif(base == 4):
    
    PASTA_RESULTADO = "OCLUSA_MANGA_RESULTADOS_" + modo + "/"
    PASTA_RESULTADOS_ARQUIVOS = "DADOS_RESULTADOS/"
    PASTA = "RESULTADOS/"
    IMAGENS_VALIDACAO_PASTA = "RESULTADOS/OCLUSA_MANGA/"
    IMAGENS_VALIDACAO_ARQUIVO = "_manga_oclusa.png"
    
    qtdImagens = 149
    
elif(base == 5):
    
    PASTA_RESULTADO = "OCLUSA_PEDUNCULO_RESULTADOS_" + modo + "/"
    PASTA_RESULTADOS_ARQUIVOS = "DADOS_RESULTADOS/"
    PASTA = "RESULTADOS/"
    IMAGENS_VALIDACAO_PASTA = "RESULTADOS/OCLUSA_PEDUNCULO/"
    IMAGENS_VALIDACAO_ARQUIVO = "_manga_pedunculo_ocluso.png"
    
    qtdImagens = 169
    
elif(base == 6):
    
    PASTA_RESULTADO = " "
    PASTA_RESULTADOS_ARQUIVOS = "DADOS_RESULTADOS/"
    PASTA = " "
    IMAGENS_VALIDACAO_PASTA = " "
    IMAGENS_VALIDACAO_ARQUIVO = "incio.png"
    
    qtdImagens = 1

In [ ]:
import cv2 
import numpy as np
import time

import PC as pc

In [ ]:
tempoInicialGeral = 0
tempoFinalGeral = 0
tempoDeterminado = 5
tempoExecutando = 0

baixo = [2, 0, 0]
alto  = [30, 255, 255]

EXTENSAO = ".txt"
INFORMACAO = "26_01_2023"

info = []

idx = 1

arquivo = open(PASTA_RESULTADOS_ARQUIVOS + INFORMACAO + "_" + PASTA_RESULTADO[:-1] + "_" + str(qtdImagens) + EXTENSAO, "w")

arquivo.write(PASTA_RESULTADOS_ARQUIVOS + INFORMACAO + "_" + PASTA_RESULTADO[:-1] + "_" + str(qtdImagens) + "\n\n")

#print(PASTA_RESULTADOS_ARQUIVOS + INFORMACAO + "_" +  PASTA_RESULTADO[:-1] + "_" + str(qtdImagens) + EXTENSAO)

sensibilidade = []
mediaHistograma = []

print("Executando...\n\n")

tempoInicialGeral = time.time()

while(idx <= qtdImagens):
            
    img = cv2.imread(IMAGENS_VALIDACAO_PASTA + str(idx) + IMAGENS_VALIDACAO_ARQUIVO)
    
    #print(IMAGENS_VALIDACAO_PASTA + str(idx) + IMAGENS_VALIDACAO_ARQUIVO)
        
    image_np = np.array(img)
    
    category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')
        
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor) 

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()
    imagem_pedunculo = image_np.copy()
    
    detection_threshold = 0.7

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                instance_masks=detections.get('detection_masks'),
                use_normalized_coordinates=True,
                max_boxes_to_draw=1,
                min_score_thresh=.8,
                skip_scores= False,
                skip_labels= True,
                #skip_boxes= False,
                agnostic_mode=True)
  
    scores = list(filter(lambda x: x> detection_threshold, 
    detections['detection_scores']))
    boxes = detections['detection_boxes'][:len(scores)]
    classes = detections['detection_classes'][:len(scores)]
    
    width = image_np_with_detections.shape[1]
    height = image_np_with_detections.shape[0]
    
    sensibilidade.append(scores)
    
    if(len(scores) > 0):

        try:

            for idxb, box in enumerate(boxes):

                roi = box * [height, width, height, width]

                xt = roi[1] #- 120     #linha esquerda
                xb = roi[3]           #linha direita
                yt = roi[0] #- 130     #linha superior
                yb = roi[2]           #linha inferior

                #Chama a função que prevê a área do pedúnculo
                areaPedunculo = pc.preveAreaPedunculo(xt, yt, xb, yb, 0.3, 0.5, -10)

                x1, y1, x2, y2 = areaPedunculo[0], areaPedunculo[1], areaPedunculo[2], areaPedunculo[3]

                #Marca um quadrado na região do pedúnculo        
                cv2.rectangle(image_np_with_detections, (x1,y1), (x2,y2), (50,0,255), 3)

                TopLeftX = x1
                TopLeftY = y2

                #Faz o corte na área prevista do pedúnculo
                corteAreaPedunculo = imagem_pedunculo[y2:y1, x1:x2]  

                #cv2.imwrite(" " + str(idx) + "_ponto_3_cadidatos.jpg", corteAreaPedunculo) 

                #Chama a função para encontrar o ponto de corte        
                coordenadaFinal = pc.coordenadaPontoFinal(corteAreaPedunculo,
                                                              baixo,
                                                              alto,
                                                              TopLeftX, 
                                                              TopLeftY, 
                                                              "Kmeans", 
                                                              7,
                                                             2)

                pontoX, pontoY, areaSegmentada, pontosCandidatos, valorHue = coordenadaFinal[0], coordenadaFinal[1], coordenadaFinal[2], coordenadaFinal[3], coordenadaFinal[4]

                mediaHistograma.append(valorHue)
                
                cv2.circle(image_np_with_detections, (pontoX, pontoY), 10, (0,0,255), -1)

                cv2.line(image_np_with_detections, (pontoX, pontoY), (int((xt+xb)/2), int(yt)), (0,255,0), 3)

                cv2.circle(image_np_with_detections, (int((xt+xb)/2), int(yt)), 10, (255,0,90), -1)

                #cv2.imwrite(PASTA_RESULTADO + str(idxb) + "_area_1_pedunculo.jpg", corteAreaPedunculo)        
                #cv2.imwrite(PASTA_RESULTADO + str(idxb) + "_area_2_segmentada.jpg", areaSegmentada)    
                
                #cv2.imwrite("RESULTADOS/" + PASTA_RESULTADO + str(idx) + "_" + str(idxb) + "_ponto_cadidatos.jpg", pontosCandidatos)
                
            cv2.imwrite(PASTA + PASTA_RESULTADO + str(idx) + "_ponto_final.jpg", image_np_with_detections)
            #print(PASTA + PASTA_RESULTADO + str(idx) + "_ponto_final.jpg", image_np_with_detections)
            
            print("{} - Fruta identificada - HUE: {}\n".format(idx, valorHue))
            
            arquivo.write(str(idx) + "," + str("1") + "\n")
            
            info.append((idx, 1, valorHue))

            idx = idx + 1
        
        except:

            print("{} - Fruta muito proxima\n".format(idx))
            
            info.append((idx, -2, 0))
            
            arquivo.write(str(idx) + "," + str("-2") + "\n")

            cv2.imwrite(PASTA + PASTA_RESULTADO + str(idx) + "_ponto_final.jpg", image_np_with_detections)

            idx = idx + 1
            
            tempoExecutando = 0
            
    else:
        
        print("{} - Fruta nao identificada\n".format(idx))
        
        info.append((idx, 0, 0))
        
        arquivo.write(str(idx) + "," + str("0") + "\n")
        
        idx = idx + 1
        
tempoFinalGeral = time.time()

arquivo.write("\n\nTempo de execução: " + str(tempoFinalGeral - tempoInicialGeral))
arquivo.close()

print("\n\nFinalizando...")

# Calcular Resultados

In [ ]:
def metricas(metri):
    
    soma = 0

    for i in range(len(info)):

        if(info[i][1] == metri):

            print(info[i])

            soma = soma + 1

    return soma

arraySensibilidade = []

for i in range(len(sensibilidade)):
       
    for j in range(len(sensibilidade[i])):

        arraySensibilidade.append(sensibilidade[i][j])

mediaSensibilidade = np.mean(arraySensibilidade)

print("Fruta identificada: {}\n".format(metricas(1)))

print("Fruta nao identificada: {}\n".format(metricas(0)))

print("Frutas proximas da camera: {}\n".format(metricas(-2)))


print("Media da sensibilidade: {:.2f}".format(mediaSensibilidade * 100))

print("Tempo de execucao: {}\n\n".format((tempoFinalGeral - tempoInicialGeral)))

print(IMAGENS_VALIDACAO_PASTA)

In [ ]:
for i in range(len(info)):
    
    print(info[i])